In [2]:
!pip install fake_useragent

# Código necessário para página liberar o conteúdo do html

In [65]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd
import time

# Defina um User-Agent legítimo
ua = UserAgent()
headers = {'User-Agent': ua.random}

# Adicione headers comuns
headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
headers['Accept-Language'] = 'pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3'
headers['Accept-Encoding'] = 'gzip, deflate, br'

# Faça a requisição
response = requests.get('https://www.transfermarkt.com.br/campeonato-brasileiro-serie-a/schiedsrichter/wettbewerb/BRA1/ajax/yw1/saison_id/all/plus//page/3', headers=headers)

# Adicione um tempo de espera
time.sleep(2)

# Direcionando para os links

In [66]:
soup = BeautifulSoup(response.text, 'lxml')
inicio = soup.find('div', attrs={'class':'row'})
page = inicio.find('div', attrs={'class':'pager'})
pages = page.select('ul li a')

# Pagando o link de cada página e pegando a página de cada Árbitro

In [ ]:
lista = []

for page in pages:
  time.sleep(5)
  ua = UserAgent()
  headers = {'User-Agent': ua.random}

  # Adicione headers comuns
  headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
  headers['Accept-Language'] = 'pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3'
  headers['Accept-Encoding'] = 'gzip, deflate, br'

  # Faça a requisição
  url = f'https://www.transfermarkt.com.br{page.get("href")}'
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, 'lxml')

  body = soup.select('div.large-8.columns div.responsive-table tbody tr.odd')
  body2 = soup.select('div.large-8.columns div.responsive-table tbody tr.even')

  # print(body2)
  for element in body2:
    link = element.select('a')
    even = f'https://www.transfermarkt.com.br{link[1].get("href")}'
    # print(even)
    ########## Adicionando módulo #########
    response2 = requests.get(even, headers=headers)
    soup3 = BeautifulSoup(response2.text, 'lxml')

    nome = soup3.select('main#tm-main div.row div.large-8.columns div.box div.info-table.info-table--equal-space span.info-table__content.info-table__content--bold')

    arbitro = nome[0].text.strip()

    if arbitro == 'Brasil' or '/' in arbitro:
      nome = soup3.select('main#tm-main header div.data-header__headline-container h1')
      idade = soup3.select('main#tm-main header div.data-header__info-box span.data-header__content')
      arbitro = nome[0].text.strip()
      print(arbitro)
      print(idade[0].get_text(strip=True))
      lista.append({'Nome': arbitro, 'Idade':idade[0].get_text(strip=True)})

    else:

      idade = soup3.select('main#tm-main header div.data-header__info-box span.data-header__content')
      print(arbitro)
      print(idade[0].get_text(strip=True))

      lista.append({'Nome': arbitro, 'Idade':idade[0].get_text(strip=True)})


  for element in body:
    link = element.select('a')
    odd = f'https://www.transfermarkt.com.br{link[1].get("href")}'
    # print(odd)

    ########## Adicionando módulo #########
    response2 = requests.get(odd, headers=headers)
    soup3 = BeautifulSoup(response2.text, 'lxml')

    nome = soup3.select('main#tm-main div.row div.large-8.columns div.box div.info-table.info-table--equal-space span.info-table__content.info-table__content--bold')


    arbitro = nome[0].text.strip()

    if arbitro == 'Brasil' or '/' in arbitro:
      nome = soup3.select('main#tm-main header div.data-header__headline-container h1')
      idade = soup3.select('main#tm-main header div.data-header__info-box span.data-header__content')
      arbitro = nome[0].text.strip()
      print(arbitro)
      print(idade[0].get_text(strip=True))
      lista.append({'Nome': arbitro, 'Idade':idade[0].get_text(strip=True)})

    else:

      idade = soup3.select('main#tm-main header div.data-header__info-box span.data-header__content')
      print(arbitro)
      print(idade[0].get_text(strip=True))

      lista.append({'Nome': arbitro, 'Idade':idade[0].get_text(strip=True)})


In [88]:

df = pd.DataFrame(lista)



In [89]:
df


,Nome,Idade
0,Wilton Pereira Sampaio,28/12/1981 (42)
1,Héber Roberto Lopes,13/07/1972 (52)
2,Raphael Claus,06/09/1979 (45)
3,Luiz Flávio de Oliveira (SP),Brasil
4,Wagner do Nascimento Magalhães,Brasil
...,...,...
268,Felipe Duarte Varejão,Brasil
269,Renán Roberto de Souza,Brasil
270,José Claudio Rocha Filho,Brasil
271,Adriano Barros,31/12/1987 (36)


In [90]:
df = df.replace('Brasil', 0)

In [92]:
# df.duplicated().sum()
df = df.drop_duplicates()

In [ ]:
df

In [94]:
df.to_excel('arbitros_prontos.xlsx', index=False)
